In [1]:
# Move to local script path to avoid problem during librairies loading

In [2]:
cd .

/var/www/Kerlandrier/scraping/2024_culturel_rosporden


In [3]:
import sys
import os
import git
import pandas as pd

# Ajoute le dossier "ressources" au sys.path
git_root = git.Repo(search_parent_directories=True).working_tree_dir
sys.path.insert(0,   os.path.abspath(  os.path.join(  git_root,'resources/python' ) ) )

from utils import *
from scraping_utils import *
from HttpRequests import *
from getAaLocation import get_or_create_oa_location
from slugify import slugify

In [4]:
access_token = retrieve_access_token(secret_key)

In [5]:
agenda_URL= "https://centreculturelrosporden.bzh/categorie-spectacle/saison/"
JsonFolder= git_root + "/scraping/2024_culturel_rosporden/"
JsonName = "eventsURL_centre_culturel.json"
columnUrlName="UrlEvent"        # Nom de la clé contenant les URL des events
# locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
# DEFAULT_TIME_START = "20:00:00"
# DEFAULT_TIME_END = "22:00:00"

In [6]:
html_doc = requests.get(agenda_URL, timeout=getTimeout).text
parsed_html = BeautifulSoup(html_doc, features="lxml")

In [7]:
# Selector pour le l'URL de chaque Event
selector=".col-md-4.col-md-offset-0 div a"
extractedALinks = parsed_html.select(selector)

In [8]:
# Extrait de la liste des URL pour vérification
print(len(extractedALinks))
linksList=[]
for el in extractedALinks:
    linksList.append(el.get('href'))
print(linksList[0])

9
https://centreculturelrosporden.bzh/spectacles/soiree-de-concerts-rap-electro-don-gabo-et-kreg-el-maout-iffick-et-soeig/


In [9]:

# Mise à jour des URLs si un scrap existe déjà
if os.path.exists(JsonFolder + JsonName):
    df_existing = pd.read_json(JsonFolder + JsonName)
    if columnUrlName in df_existing.columns:
        existingUrls = df_existing[columnUrlName].tolist()
        
        urlsManquantes = [url for url in linksList if url not in existingUrls]
        for url in urlsManquantes:
            print(f"L'URL' '{url}' n'est pas présent dans le fichier JSON du précédent Scrap ! Il sera ajouté.")
        
        # Ajouter les nouvelles entrées si des noms sont manquants
        if urlsManquantes:
            dfNewUrls = pd.DataFrame(urlsManquantes, columns=[columnUrlName])
            df_Events = pd.concat([df_existing, dfNewUrls])
        else:
            df_Events = df_existing
            dfNewUrls = pd.DataFrame()
    # Backup JSON File
    dateNow=datetime.datetime.now().strftime("%Y-%m-%d_%H:%M")
    os.rename(JsonFolder + JsonName, JsonFolder + dateNow + "_"+ JsonName + ".BAK")
# Sinon création de la liste des events
else: df_Events = pd.DataFrame(linksList, columns=[columnUrlName])


In [15]:
# Sauvegarde en JSON

## NB:
## json.dump has better formatting & don't escape / in \/  but it doesn't  replace NaN by null
df_Events.to_json(JsonFolder + JsonName, orient="records", force_ascii=False)
with open(JsonFolder + JsonName) as json_file:
    events = json.load(json_file)
json.dump(events, open(JsonFolder + JsonName, "w"), indent=4)


In [11]:
## To DO :  filtre des events selon critères
##          customisation selon les besoins
# for event in list_of_events_url:
#     if event.get('url') == "machin"


In [12]:
# Vars to collect data from an single event webpage

nameSelector = "#page > section.jumbotron.principal > div h1"
longDescriptionSelector = "#page > section.jumbotron.principal > div > div.col-md-9.col-sm-9.col-xs-12.col-md-offset-0 > p"
descriptionSelector ="#page > section.jumbotron.principal > div > div.col-md-9.col-sm-9.col-xs-12.col-md-offset-0 > p:nth-child(3)"
dateSelector = "#page > section.jumbotron.principal > div h3"

In [ ]:
for index, event in df_Events.iterrows():
    try :
        print(event['UrlEvent'])
        event['name']= get_string_from_webpage(event['UrlEvent'], nameSelector)
        print(event['name'])
        event['longDescription']=get_string_from_webpage(event['UrlEvent'],longDescriptionSelector) # Description longue
        event['description']=get_string_from_webpage(event['UrlEvent'],descriptionSelector) # Description courte
        # Date et heure
        dateText=get_string_from_webpage(event['UrlEvent'],dateSelector) # Date et heure de début
        date_begin=get_datetime_from_text(dateText)
        event['date_begin']=date_begin
        duree= "2h"
        date_end=get_end_date(date_begin, duree)
        event['date_end']=date_end
        locationText= "Centre culturel de Rosporden"
        location_uid =  get_or_create_oa_location(locationText)
    except Exception as e:
        print(e)
    
    # imgTag= 'data-lazy-src'
    # imageURL = get_image_from_webpage(url = event['UrlEvent'],
    #                         selector = imageSelector,
    #                         imgTag= imgTag,
    #                         path = None,
    #                         )
    

https://centreculturelrosporden.bzh/spectacles/soiree-de-concerts-rap-electro-don-gabo-et-kreg-el-maout-iffick-et-soeig/

start_date is None
https://centreculturelrosporden.bzh/spectacles/emily-loizeau-la-souterraine/

start_date is None
https://centreculturelrosporden.bzh/spectacles/titi-tombe-titi-tombe-pas-compagnie-pascal-rousseau/

start_date is None
https://centreculturelrosporden.bzh/spectacles/on-disait-quon-faisait-la-fete-fanny-ruwet/

start_date is None
https://centreculturelrosporden.bzh/spectacles/numero-dys-compagnie-la-rigole/

start_date is None
https://centreculturelrosporden.bzh/spectacles/je-suis-tigre-groupe-noces-florence-bernad/

start_date is None
https://centreculturelrosporden.bzh/spectacles/vivaldi-et-venise-ensemble-amalthee/

start_date is None
https://centreculturelrosporden.bzh/spectacles/truc/



In [ ]:
# TO DO : Check data values :
# - date_begin < date_end
# - date_end & date_begin in 2024 or 2025
# - location_uid not null
# - len(name, summary, description )> 5


In [ ]:
# Post events
for index, event in df_Events.iterrows():
        if date_begin > datetime.datetime.now():
                eventOA= {
                        "uid-externe": "import" + "-" + slugify(event['name']),
                        "title": { "fr": event['name'] ,
                                "en": event['name'] } ,
                        "summary": { "fr": event['summary'] },
                        "description": { "fr": event['description'], "en": event['description']  },
                        "locationUid": int(event['location_uid']),
                        "links": event['UrlEvent'],
                        "longDescription": event['longDescription'] ,   
                        "keywords": { "fr": ["CapDanseEvent", "CapDanse", "danse"] },
                        "timings": [
                                {
                                "begin": date_begin.isoformat(),
                                "end": date_end.isoformat()
                                },
                                ],
                        "attendanceMode": 3,
                        "onlineAccessLink": event['UrlEvent']
                }
                
                if event['UID'] is not None:
                        UID = patch_event(access_token,event['UID'],eventOA)
                else:
                        UID = create_event(access_token,eventOA)
                        event['UID'] = UID

# Save result
df_Events.to_json(JsonFolder + JsonName, orient="records", force_ascii=False)
with open(JsonFolder + JsonName) as json_file:
        events = json.load(json_file)
json.dump(events, open(JsonFolder + JsonName, "w"), indent=4)
